## Azimuthal Dust Profiles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as c
import astropy.units as u
import argparse

import azimuthal_profile as az
import dsharp_opac as op
from scipy.interpolate import interp2d

if az.is_interactive():
    get_ipython().run_line_magic('matplotlib', 'inline')

au = c.au.cgs.value
c_light = c.c.cgs.value
jy_as = (1. * u.Jy / u.arcsec).cgs.value

In [ ]:
RTHF   = argparse.RawTextHelpFormatter
PARSER = argparse.ArgumentParser(description='Code to calcuate azimuthal equilibrium size distributions based on Birnstiel et al. 2013.', formatter_class=RTHF)
PARSER.add_argument('-p', '--do-plots', help='produce plots', action='store_true', default=False)
PARSER.add_argument('-r', '--rplot', help='at which radius (in au) to make plots', type=float, default=100.)
PARSER.add_argument('-o', '--output', help='file name for output', default='modeloutput.txt', type=str)

PARSER.add_argument('--agas', help='gas azimuthal contrast', type=float, default=1.2)
PARSER.add_argument('--d2g', help='dust-to-gas mass ratio', type=float, default=0.01)
PARSER.add_argument('--alpha', help='turbulence parameter', type=float, default=0.001)
PARSER.add_argument('--sigc', help='gas density scale', type=float, default=100.)
PARSER.add_argument('--v-frag', help='fragmentation velocity', type=float, default=1000.)
PARSER.add_argument('--sigma-y', help='azimuthal bump extend in degree', type=float, default=10.)
PARSER.add_argument('--a-max', help='maximum particle size\n- can be a float (a_max in cm)\n- "None" makes B11 size distribution\n- can also evaluate expressions like, 0.1 * 100 * au /r', default=None)

if not az.is_interactive():
    ARGS  = PARSER.parse_args()
else:
    ARGS  = PARSER.parse_args([])
    print('interactive -> using default arguments')

### Set up model parameters of a radial gas profile

In [ ]:
na          = 100
ny          = 400
rho_s       = 1.67

sigma_y_deg = ARGS.sigma_y     # azimuthal bump extent in degree
A_gas       = ARGS.agas    # gas density contrast (peak-to-valley)
d2g         = ARGS.d2g
alpha       = ARGS.alpha
v_frag      = ARGS.v_frag
sigc        = ARGS.sigc
M_star      = c.M_sun.cgs.value
r_plot      = ARGS.rplot * au  # at which radius to create the plots
do_plot     = ARGS.do_plots  # to plot or not to plot

sigma_y = sigma_y_deg / 180 * np.pi
r       = np.arange(5, 400, 5) * au  # radial grid
Y       = r[:, None] * np.linspace(- np.pi, np.pi, ny)  # azimuth grid
sig_g   = sigc / (r / au)                     # avg. gas surface density
sig_d   = d2g * sig_g

# maximum particle size, which can be
# float: same maximum particle size everywhere
# array: different particle size at every radius
# None: will construct fragmentation/coagulation size distribution

a_max   = ARGS.a_max

if type(ARGS.a_max) is str:
    try:
        a_max = float(a_max)
        print(f'using a_max = {a_max:.3g}')
    except:
        print('evaluating a_max function')
        a_max = eval(a_max)

# size grid: at least to a_max or 10 if a_max is none
if a_max is None:
    A = np.logspace(-5, 1, na)
else:
    A = np.logspace(-5, np.log10(np.max(a_max)), na)

# Nienke: temperature has minor influence, so changes due to changes
# in R is primarily due to sigma_gas, which is scaled with R => change sig_c
T       = 200 * (r / au)**-0.5

sig_g_2D = 1 + (A_gas - 1) * np.exp(- Y**2 / ((2 * (r * sigma_y))**2)[:, None])
sig_g_2D *= (sig_g / sig_g_2D.mean(-1))[:, None]

### Get the equilibrium dust distribution

In [ ]:
if a_max is not None:
    print('a_max given -> making MRN size distribution')

    # make an MRN distribution up to a_max

    distri = A[None, :]**0.5 * np.ones([len(r), 1])
    mask = A[None, :] > (a_max * np.ones_like(r))[:, None]
    distri[mask] = 1e-100
    distri *= (sig_d / distri.sum(-1))[:, None]
    distri = distri.T

    if do_plot:
        f, ax = plt.subplots()
        ax.pcolormesh(r / au, A, np.log10(distri + 1e-10))
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_title('size distribution')
        ax.set_xlabel('$r$ [au]')
        ax.set_ylabel('paraticle size [cm]')
        f.savefig('size_distribution.pdf', transparent=True)
else:
    print('no a_max given -> using equilibrium size distribution')
    distri = sig_d.copy()

In [ ]:
sig_d_2D = az.make_azim_distri(
    r,
    Y,
    A,
    T,
    sig_g_2D,
    distri,
    alpha,
    v_frag,
    rho_s,
    M_star,
    )

### Plot the azimuthal distribution of grains

In [ ]:
if do_plot:
    ir = r.searchsorted(r_plot)

    f, ax = plt.subplots()

    y = Y[ir, :] / r[ir] * 180 / np.pi  # azimuthal grid in degree

    ax.plot(y, sig_g_2D[ir, :] / 100, 'k-', label='gas / 100')

    for ia in np.arange(na)[::na // 6]:
        ax.plot(y, sig_d_2D[ir, :, ia], 'x-', label=f'{A[ia]:.2g} cm')

    ax.set_xlim(-45, 45)
    ax.set_xlabel('azimuth [degree]')
    ax.set_ylabel(r'$\Sigma_\mathrm{d}$ [g cm$^{-2}$]')
    ax.set_title(f'at {r[ir] / au:.2f} au')
    ax.legend()
    f.savefig('azimuthal_density.pdf', transparent=True)

### Plot the contrast as function of particle size

In [ ]:
if do_plot:
    ir = r.searchsorted(r_plot)

    St = A * rho_s * np.pi / (2. * sig_g[ir])
    A_d = sig_d_2D.max(-2)[ir, :] / sig_d_2D.min(-2)[ir, :]
    A_d_ana = A_gas * np.exp(St * (A_gas - 1) / (A_gas * alpha))

    mask = sig_d_2D[ir].mean(0) > 1e-10

    f, ax = plt.subplots()
    ax.loglog(St[mask], A_d[mask], label='from profile')
    ax.loglog(St, A_d_ana, 'k--', label='analytical')

    ax.set_xlabel('Stokes number')
    ax.set_ylabel('Dust contrast')
    ax.set_title(f'at {r[ir] / au:.2f} au')
    ax.legend()
    ax.set_ylim(1e0, 1e10)
    f.savefig('contrast_curve.pdf', transparent=True)

### Plot the particle size distribution

In [ ]:
if do_plot:
    ir = r.searchsorted(r_plot)

    distri = sig_d_2D[ir, :, :].mean(0)

    f, ax = plt.subplots()

    ax.loglog(A, distri)
    ax.set_ylim(distri.max() * 1e-5, 2 * distri.max())
    ax.set_xlabel('particle size [cm]')
    ax.set_title(f'at {r[ir] / au:.2f} au')
    ax.set_ylabel(r'$\sigma$ [g cm$^{-2}$]')
    f.savefig('size_distri_slice.pdf', transparent=True)

### Read opacity and interpolate on our grid

In [ ]:
lam_obs = np.array([0.13, 0.9])  # our obervational wavelength
n_lam = len(lam_obs)
nu_obs  = c_light / lam_obs

In [ ]:
with np.load(op.get_datafile('default_opacities_smooth.npz')) as fid:
    a_opac   = fid['a']
    lam_opac = fid['lam']
    k_abs    = fid['k_abs']
    k_sca    = fid['k_sca']
    g        = fid['g']
    rho_s    = fid['rho_s']

f_kappa = interp2d(np.log10(lam_opac), np.log10(a_opac), np.log10(k_abs))
k_a = 10.**f_kappa(np.log10(lam_obs), np.log10(A))

In [ ]:
if do_plot:
    f, ax = plt.subplots()
    for ilam in range(n_lam):
        ax.loglog(A, k_a[:, ilam], label=f'$\\lambda = {10 * lam_obs[ilam]:.2g}$ mm')
    ax.set_xlabel('particle size [cm]')
    ax.set_ylabel('$\\kappa_\\mathrm{abs}$ [cm$^2$/g]')
    ax.legend()
    f.savefig('opacity.pdf', transparent=True)

### Calculate intensity profile

In [ ]:
# B_nu has shape (n_wavelength, n_radii)
# tau and I_nu should have shape (n_wavelength, n_radii, n_azimuth)
B_nu = az.planck_B_nu(nu_obs, T).T
tau = (sig_d_2D[None, ...] * k_a.T[:, None, None, :]).sum(-1)
I_nu = B_nu[:, :, None] * (1 - np.exp(-tau))

In [ ]:
if do_plot:
    ir = r.searchsorted(r_plot)

    f, ax = plt.subplots()

    for ilam in range(n_lam):
        print(f'Intensity contrast (max/min) at {10 * lam_obs[ilam]:3.1f} mm = {I_nu[ilam, ir, :].max()/I_nu[ilam, ir, :].min():.3g}')

        ax.semilogy(y, I_nu[ilam, ir, :] / jy_as, label=f'$\\lambda = {10 * lam_obs[ilam]:3.1f}$ mm')

    ax.set_xlabel('azimuth [degree]')
    ax.set_ylabel('$I_\\nu$ [Jy/arsec]')
    ax.set_title(f'at {r[ir] / au:.2f} au')
    ax.legend()
    f.savefig('intensity_profiles.pdf', transparent=True)

## Create output

In [ ]:
with open(ARGS.output, 'w') as f:
    f.write(f'# Model with Agas={A_gas:.3g}, alpha={alpha:.3g}, d2g={d2g:.3g}, Sigc={sigc:.3g}, amax={ARGS.a_max}\n')
    f.write('# Radius  \tSiggas(R)  \tContrastmm  \tContrastcm \n')
    np.savetxt(f, np.vstack((r / au, sig_g, I_nu.max(-1) / I_nu.min(-1))).T, delimiter='\t', fmt='%2.2e')